In [ ]:
#https://www.analyticsvidhya.com/blog/2021/12/text-classification-using-bert-and-tensorflow/?utm_source=related_WP&utm_medium=https://www.analyticsvidhya.com/blog/2021/12/text-classification-of-news-articles/
import os
!pip install tensorflow-addons
!pip install transformers datasets sklearn torch
from scipy.stats import randint
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

%matplotlib inline

  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1/data_set.csv')

print(df.shape)

(12680, 10)


# **Pre-processing:**

In [ ]:
df = df.dropna()

df = df.dropna(subset=['Final_Category'])

In [ ]:
df = df.drop_duplicates(subset='Title', keep='first')

df.head(2)

,SL no,Title,Link,Date,Double_Category,Priority_Category,Category,Final_Category,Content,Summary
0,1,British LAPSUS$ Teen Members Sentenced for Hig...,https://thehackernews.com/2023/12/british-laps...,"Dec 24, 2023",Cyber Crime / Data Breach,Data Breach,Cyber Attack,Cyber Attack,Two British teens part of the LAPSUS$ cyber cr...,two teens from the gang have been sentenced to...
1,2,Operation RusticWeb: Rust-Based Malware Target...,https://thehackernews.com/2023/12/operation-ru...,"Dec 22, 2023",Malware / Cyber Threat,Cyber Threat,Malware,Malware,Indian government entities and the defense sec...,a phishing campaign has been engineered to dro...


Creating a duplicate dataset:






In [ ]:
df1 = df[['Title', 'Date', 'Link', 'Final_Category', 'Content', 'Double_Category']].copy()

df1 = df1[pd.notnull(df1['Final_Category'])]

df1.columns = ['Title', 'Date', 'Link', 'Category', 'Content', 'Initial_Lable']

print(df1.shape)


(12317, 7)


In [ ]:
double_unique_labels = df1['Initial_Lable'].unique()

unique_labels = df1['Category'].unique()


print(unique_labels)

print(len(unique_labels))

print('===================================')

print(double_unique_labels)

print(len(double_unique_labels))

['Cyber Crime / Data Breach' 'Malware / Cyber Threat'
 'Skimming / Web Security' ... 'Online Scam / Cyber Threat'
 'Skimming / Banking Trojan' 'Network Security / Threat Analysis']
4595


In [ ]:
df1['Content_Length'] = df1['Content'].apply(len)

shortest_content = df1.loc[df1['Content_Length'].idxmin()]

longest_content = df1.loc[df1['Content_Length'].idxmax()]

df1 = df1.drop('Content_Length', axis=1)


print("Shortest Content length:")
print(len(shortest_content['Content']))
print(shortest_content['Content'])

print('-------------------------------------')

print("\nLongest Content length:")
print(len(longest_content['Content']))


Shortest Content length:
2



-------------------------------------

Longest Content length:
33578


In [ ]:
df1['Content_Length'] = df1['Content'].apply(len)

min_length = 200
max_length = 33578

for index, row in df1.iterrows():
    if row['Content_Length'] < min_length or row['Content_Length'] > max_length:
        df1 = df1.drop(index)

df1 = df1.drop('Content_Length', axis=1)

print(df1.shape)


(12108, 7)


In [ ]:
category_mapping = {
    'Cyber Attack': 0,
    'Malware': 1,
    'Vulnerability': 2,
    'Data Breach': 3
}

df1['category_id'] = df1['Category'].map(category_mapping)

mapping_series = pd.Series(category_mapping)
print(mapping_series)


Cyber Attack     0
Malware          1
Vulnerability    2
Data Breach      3
dtype: int64


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

df1['Clean_Content'] = df1['Content'].apply(preprocess_text)

# **System internal code:**

-> Previous query (initially empty)

In [ ]:
previous_query = ""

->user given query

In [ ]:
active_query = "Give analysis about the threat of Cyber Security"

-> User id and Session start time

In [ ]:
import uuid
from datetime import datetime

def initialize_session():
    user_id = str(uuid.uuid4())

    session_start_time = datetime.now()

    print(f"User ID: {user_id}")
    print(f"Session started at: {session_start_time}")

    return user_id, session_start_time

def session_duration(session_start_time):
    current_time = datetime.now()

    duration = current_time - session_start_time

    print(f"Active session duration: {duration.seconds} seconds")

user_id, session_start_time = initialize_session()



User ID: 7504f317-7be9-40b5-8237-45ac6fbbd033
Session started at: 2024-10-10 16:01:16.100964


------------------------------------------------------------------------------

# **-> Module 1 (query checking and correcting the word)**

In [ ]:
!pip install symspellpy
!python -m spacy download en_core_web_sm
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=6314024be35fdd59267d1053d09fddaecaa82ae0a8b81a8601049e062d904557
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install transformers
!pip install torch
!pip install spacy symspellpy language-tool-python transformers torch
!pip install pyspellchecker langdetect
!pip install langid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=6395a9bbb620236a7c6bf66dec517eb291f75c704ebe17a9ab69600a2941ceb3
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [ ]:
import language_tool_python
import langid
from langdetect import detect, LangDetectException
import logging

tool = language_tool_python.LanguageTool('en-US')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpuovvuo6z.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


In [ ]:
def correct_grammar_spelling(text):
    try:
        matches = tool.check(text)
        corrected_text = language_tool_python.utils.correct(text, matches)
        return corrected_text
    except Exception as e:
        logging.error(f"Error during grammar correction: {e}")
        return None

def analyze_query(query):
    try:
        lang = detect(query)
    except LangDetectException:
        print("Could not detect language. Please use English words.")
        return None
    except Exception as e:
        logging.error(f"Error during language detection: {e}")
        return None

    logging.info(f"Detected language: {lang}")

    if lang != 'en':
        print(f"Detected language: {lang}. Please use English words or complete the sentence.")
        return None

    corrected_text = correct_grammar_spelling(query)
    return corrected_text

current_query = analyze_query(active_query)

if current_query:
    print(f"Active Query: {active_query}")
    print(f"Corrected Query: {current_query}")
else:
    print("The query was not in English or couldn't be processed.")

Active Query: Give analysis about the threat of Cyber Security
Corrected Query: Give analysis of the threat of Cybersecurity


--------------------------------------------------------------------------------

# **-> Module 2 (queries sequence maintainance)**

In [ ]:
PRONOUNS = ["it", "this", "that", "these", "those", "he", "she", "they", "his", "her", "their", "its"]

QUESTION_WORDS = {"what", "how", "why", "which", "where", "when", "who", "whose", "whom"}

DESCRIPTIVE_WORDS = {'patching', 'influences', 'malfunctions', 'authentication issue', 'importance', 'role', 'concerns', 'scope', 'dangers', 'privacy concern', 'consequences', 'purposes', 'security flaw', 'data vulnerability', 'technical difficulty', 'NLP challenge', 'problem', 'issues', 'applications', 'availability', 'exploit', 'solutions', 'exploits', 'risk factor', 'buffer overflow', 'mitigation', 'attack surface', 'necessities', 'detection rate', 'injection', 'objectives', 'integrity', 'drawbacks', 'effects', 'false positive', 'benefits', 'issue', 'significance', 'considerations', 'limitation', 'security threat', 'weaknesses', 'danger', 'problems', 'impact', 'opportunities', 'backdoor', 'risks', 'implications', 'firewall', 'impacts', 'limitations', 'potential', 'exposures', 'confidentiality', 'strengths', 'relevance', 'machine learning issue', 'penetration', 'challenges', 'barriers', 'glitches', 'system performance', 'system breach', 'feasibility', 'threats', 'risk mitigation', 'vulnerabilities', 'penetration test', 'response', 'data leak', 'breaches', 'outcomes', 'bug', 'disadvantages', 'defense', 'security risk', 'factors', 'constraints', 'accuracy issue', 'security gap', 'AI challenge', 'advantages', 'AI limitation', 'priorities', 'issue mitigation', 'requirements', 'use'}

IRRELEVANT_WORDS = {'tell','the', 'example', 'best', 'bit', 'note', 'something', 'thing', 'output', 'the concept', 'input', 'type', 'subject', 'detail', 'result', 'process', 'anything', 'another', 'whatever', 'other', 'topic', 'section', 'aspect', 'different', 'matter', 'generic', 'context', 'me', 'specific', 'particular', 'individual', 'basic', 'general', 'piece', 'information', 'content', 'various', 'text', 'element', 'page', 'concept', 'item', 'an example', 'certain', 'more'}


KNOWN_MWES = {
    "cyber attack", "data breach", "network security", "ransomware attack", "phishing scam", "denial of service",
    "distributed denial of service", "social engineering", "firewall configuration",
    "intrusion detection", "identity theft", "vulnerability assessment", "cyber threat intelligence",
    "man-in-the-middle attack", "zero-day vulnerability", "malware infection", "command injection",
    "SQL injection", "cross-site scripting", "buffer overflow", "backdoor attack",
    "brute force attack", "credential stuffing", "password spraying", "honeypot trap",
    "advanced persistent threat", "cyber espionage", "insider threat", "session hijacking",
    "keylogging", "rootkit", "spyware", "adware", "botnet", "cloud misconfiguration",
    "remote code execution", "cryptojacking", "data exfiltration", "exploit kit",
    "sandbox evasion", "cyber warfare", "ethical hacking", "penetration testing", "fuzz testing",
    "network intrusion", "black hat hacking", "phishing detection", "security patch",
    "two-factor authentication", "zero trust architecture", "cryptography", "public key infrastructure",
    "machine learning", "cyber security", 'data loss', 'validation loss', 'data breach softwares', 'malware attacks'
}


In [ ]:
import re
import spacy
import torch
from transformers import BertTokenizer, BertForNextSentencePrediction
from difflib import SequenceMatcher

nlp = spacy.load("en_core_web_sm")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [ ]:
def detect_and_preserve_mwes(query):
    mwes = set()
    for mwe in KNOWN_MWES:
        if mwe in query.lower():
            mwes.add(mwe)
            query = query.lower().replace(mwe, "")
    return mwes, query

def extract_topics_using_dependencies(query):
    mwes, processed_query = detect_and_preserve_mwes(query)
    doc = nlp(processed_query)
    topics = set()

    for token in doc:
        if token.pos_ in ("NOUN", "PROPN") and token.text.lower() not in DESCRIPTIVE_WORDS and token.text.lower() not in IRRELEVANT_WORDS:
            topics.add(token.text.strip())

    topics.update(mwes)

    return list(topics)

def expand_current_topics_with_previous(current_topics, previous_topics, current_query):
    for topic in previous_topics:
        if topic.lower() in current_query.lower() and topic.lower() not in current_topics:
            current_topics.append(topic)
    return current_topics

def detect_pronouns(query):
    doc = nlp(query)
    pronouns = [token.text for token in doc if token.pos_ == "PRON" and token.text.lower() in PRONOUNS]
    return pronouns if pronouns else None

def replace_pronouns_in_query(current_query, main_topics_str):
    placeholder_replaced = False
    for placeholder in PRONOUNS:
        if re.search(rf"\b{placeholder}\b", current_query, flags=re.IGNORECASE):
            current_query = re.sub(rf"\b{placeholder}\b", main_topics_str, current_query, flags=re.IGNORECASE)
            placeholder_replaced = True
    return current_query, placeholder_replaced

def calculate_similarity(previous_query, current_query):
    return SequenceMatcher(None, previous_query.lower(), current_query.lower()).ratio()

def calculate_nsp_score(previous_query, current_query):
    inputs = tokenizer.encode_plus(previous_query, current_query, return_tensors='pt')
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    is_next_prob = probs[0][0].item()
    return is_next_prob

def is_query_related(previous_query, current_query):

    if previous_query != "":

        previous_topics = extract_topics_using_dependencies(previous_query)
        current_topics = extract_topics_using_dependencies(current_query)

        current_topics = expand_current_topics_with_previous(current_topics, previous_topics, current_query)

        previous_topics_str = ", ".join(previous_topics)
        current_topics_str = ", ".join(current_topics)

        print(f"Previous Topics: {previous_topics_str}")
        print(f"Current Topics: {current_topics_str}")

        #1. Condition: Check if there is a keyword match and a pronoun
        detected_pronouns = detect_pronouns(current_query)
        if previous_topics and current_topics and any(topic in current_topics for topic in previous_topics) and detected_pronouns:
            print(f"Pronouns detected: {', '.join(detected_pronouns)}")
            modified_query, _ = replace_pronouns_in_query(current_query, " and ".join(current_topics))
            print("Related: First Condition - Keyword match and pronoun present.")
            return True, modified_query

        #2. Condition: If no keyword match but there is a pronoun, not related
        if previous_topics and current_topics and not any(topic in current_topics for topic in previous_topics) and detected_pronouns:
            print(f"Pronouns detected: {', '.join(detected_pronouns)}")
            modified_query, _ = replace_pronouns_in_query(current_query, " and ".join(current_topics))
            print("Not Related: Second Condition - No keyword match but pronoun detected.")
            return False, modified_query

        #3. Condition: If no keyword match, check for pronouns
        if not current_topics and detected_pronouns:
            modified_query, _ = replace_pronouns_in_query(current_query, " and ".join(previous_topics))
            print("Related: Third Condition - No keyword match but pronoun present.")
            return True, modified_query

        #4. Condition: If no pronouns and no valid topics
        if not current_topics:
            modified_query = current_query + " of " + " and ".join(previous_topics)
            print("Related: Fourth Condition - No valid topics and no pronouns.")
            return True, modified_query
        elif current_topics:
            modified_query = current_query
            print("Not Related: Fourth Condition - Valid topics present in current query.")
            return False, modified_query

        #5. Condition: If all other conditions fail, use similarity or NSP score
        similarity_score = calculate_similarity(previous_query, current_query)
        if similarity_score > 0.85:  # Similarity threshold
            modified_query = current_query
            print(f"Related: Fifth Condition - Similarity score indicates relatedness. Score: {similarity_score:.2f}")
            return True, modified_query

        print(f"Not Related: Fifth Condition - Similarity score too low. Score: {similarity_score:.2f}")
        return False, current_query

    else:
        #If previous query is empty and the current query has pronouns and keywords
        detected_pronouns = detect_pronouns(current_query)
        current_topics = extract_topics_using_dependencies(current_query)
        if detected_pronouns and current_topics:
            modified_query, _ = replace_pronouns_in_query(current_query, " and ".join(current_topics))
            print(f"Pronouns detected: {', '.join(detected_pronouns)}")
            previous_query = modified_query
            return False, modified_query
        else:
            modified_query = current_query
            previous_query = current_query
            return False, modified_query


related, modified_query = is_query_related(previous_query, current_query)

if related:
    print("The current query is related to the previous query.")
    print(f"Modified Query: {modified_query}")
    related = 'yes'

else:
    print("The current query is NOT related to the previous query.")
    print(f"Modified Query: {modified_query}")
    previous_query = ""
    related = 'no'


The current query is NOT related to the previous query.
Modified Query: Give analysis of the threat of Cybersecurity


In [ ]:
print(related)

no


---------------------------------------------------------------------------------------------

# **-> Module 3 (Top 10 Rows generation based on query)** need GPU

In [ ]:
!pip install rank-bm25 transformers gensim scikit-learn numpy pandas

need to run only one:

In [ ]:
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from gensim.utils import simple_preprocess
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

generated_title_history = []

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
bert_model = BertModel.from_pretrained('bert-large-uncased')

def tokenize_and_initialize_bm25(df1):
    tokenized_corpus = [simple_preprocess(doc) for doc in df1['Clean_Content'].dropna()]
    bm25_model = BM25Okapi(tokenized_corpus)
    return bm25_model, tokenized_corpus

bm25_model, tokenized_corpus = tokenize_and_initialize_bm25(df1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_bert_embedding(text, tokenizer, model, device):
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024, padding=True).to(device)
    outputs = model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.cpu().detach().numpy()

def bm25_pipeline_faster(bm25_model, query, df1, top_n=10, exclude_titles=None):

    tokenized_query = simple_preprocess(query)
    bm25_scores = bm25_model.get_scores(tokenized_query)

    top_indices = np.argsort(bm25_scores)[-top_n*3:][::-1]
    top_n_candidates_df = df1.iloc[top_indices][['Title', 'Clean_Content', 'Link', 'Initial_Lable', 'category_id']]

    if exclude_titles:
        top_n_candidates_df = top_n_candidates_df[~top_n_candidates_df['Title'].isin(exclude_titles)]

    return top_n_candidates_df.head(top_n)

def re_rank_with_bert(top_n_contents_df, query, tokenizer, model):

    query_embedding = get_bert_embedding(query, tokenizer, model, device)

    document_embeddings = np.array([get_bert_embedding(doc, tokenizer, model,device) for doc in top_n_contents_df['Clean_Content']])

    document_embeddings = document_embeddings.squeeze(1)

    similarities = cosine_similarity(query_embedding, document_embeddings).flatten()

    top_n_contents_df['similarity'] = similarities
    top_n_contents_df = top_n_contents_df.sort_values(by='similarity', ascending=False)

    return top_n_contents_df

def bm25_bert_hybrid_pipeline(modified_query, df1, bm25_model, related='no', top_n=10):
    if related == 'no':

        top_10_contents_df = bm25_pipeline_faster(bm25_model, modified_query, df1, top_n=top_n)

        if not generated_title_history:

            generated_title_history.extend(top_10_contents_df['Title'].tolist())
        else:

            generated_title_history.clear()
            generated_title_history.extend(top_10_contents_df['Title'].tolist())
    elif related == 'yes':

        top_10_contents_df = bm25_pipeline_faster(bm25_model, modified_query, df1, top_n=top_n, exclude_titles=generated_title_history)
        generated_title_history.extend(top_10_contents_df['Title'].tolist())

    re_ranked_results = re_rank_with_bert(top_10_contents_df, modified_query, tokenizer, bert_model)

    return re_ranked_results


top_10_contents_df = bm25_bert_hybrid_pipeline(modified_query, df1, bm25_model, related=related, top_n=10)

top_10_contents_df

,Title,Clean_Content,Link,Initial_Lable,category_id
3363,Critical Flaws in Philips TASY EMR Could Expos...,the u s cybersecurity and infrastructure secur...,https://thehackernews.com/2021/11/critical-fla...,Vulnerability / Medical Software,2
3393,Hackers Using Squirrelwaffle Loader to Deploy ...,a new spam email campaign has emerged as a con...,https://thehackernews.com/2021/10/hackers-usin...,Malware / Loader Exploits,1
5223,Get 10 Popular Books To Learn Advanced Hacking...,it should come as no surprise that cybersecuri...,https://thehackernews.com/2019/01/cybersecurit...,Cybercrime / Threat Intelligence,3
10017,Pentagon boosts contractor cybersecurity program,pentagon boosts contractor cybersecurity progr...,https://thehackernews.com/2012/05/pentagon-boo...,Cybersecurity / Government Security,2
4147,Guide: How Security Consolidation Helps Small ...,the dynamic nature of cybersecurity the change...,https://thehackernews.com/2021/02/guide-how-se...,Cybersecurity Training / IT Security,0
1417,FakeCalls Vishing Malware Targets South Korean...,an android voice phishing aka vishing malware ...,https://thehackernews.com/2023/03/fakecalls-vi...,Mobile Security / Scam Alert,3
2053,How To Build a Career as a Freelance Cybersecu...,with each passing year the cybersecurity threa...,https://thehackernews.com/2022/10/how-to-build...,Cybersecurity Training / IT Security,0
4499,New USBCulprit Espionage Tool Steals Data From...,a chinese threat actor has developed new capab...,https://thehackernews.com/2020/06/air-gap-malw...,Cyber Espionage / Cyber Attack,3
2631,Researchers Find New Malware Attacks Targeting...,an unknown advanced persistent threat apt grou...,https://thehackernews.com/2022/05/researchers-...,Malware / Cyber Espionage,0
4724,The Hacker News 2020 Cybersecurity Salary Surv...,for the first time the hacker news launches a ...,https://thehackernews.com/2019/11/cybersecurit...,Cybersecurity Training / IT Security,3


can create a csv file from these top 10:

In [ ]:
#location
file_path = '/content/drive/MyDrive/1/CSV/top_10_results.csv'

#saving the top_10_contents_df to the location
top_10_contents_df.to_csv(file_path, index=False)

------------------------------------------------------------------------------------------

# -> Module 4 (Classification)

In [ ]:
#run one time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file = '/content/drive/MyDrive/1/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
top_10_contents_df['Clean_Content_Tokens'] = top_10_contents_df['Clean_Content'].apply(word_tokenize)

def get_avg_glove_vector(tokens, embeddings_index, vector_size):
    vectors = []
    for token in tokens:
        if token in embeddings_index:
            vectors.append(embeddings_index[token])
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

vector_size = 300
top_10_contents_df['Clean_Content_Vector'] = top_10_contents_df['Clean_Content_Tokens'].apply(lambda x: get_avg_glove_vector(x, glove_embeddings, vector_size))


modified_query_tokens = word_tokenize(modified_query)

modified_query_vector = get_avg_glove_vector(modified_query_tokens, glove_embeddings, vector_size).reshape(1, -1)

clean_content_vectors = np.stack(top_10_contents_df['Clean_Content_Vector'].values)
similarity_scores = cosine_similarity(clean_content_vectors, modified_query_vector).flatten()

top_10_contents_df['similarity'] = similarity_scores

top_10_contents_df = top_10_contents_df.sort_values(by='similarity', ascending=False)

top_match = top_10_contents_df.iloc[0]

predicted_initial_label = top_match['Initial_Lable']
predicted_category_id = top_match['category_id']

print(f"Predicted Initial_Lable: {predicted_initial_label}")
print(f"Predicted category_id: {predicted_category_id}")


Most frequent category_id: 3
Predicted Initial_Lable for the query: Cybercrime / Threat Intelligence


------------------------------------------------------------------------------------

# **Module 5 (Text generation from top 10 content)**

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

#Checking if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

#loding the BART-base model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

#moving the model to the GPU
model.to(device)

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
def summarize_content_with_bart(content, query, max_length=100, min_length=30, num_beams=4, batch_size=4):

    input_texts = [f"query: {query} document: {content_part}" for content_part in content]

    inputs = tokenizer(input_texts, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(device)  #Move inputs to GPU

    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        min_length=min_length,
        temperature=0.8,
        repetition_penalty=2.9,
        num_beams=num_beams,
        length_penalty=2.0,
        early_stopping=True
    )

    summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

    return summaries


relevant_texts = top_10_contents_df['Clean_Content'].tolist()

batch_size = 4
summarized_texts = []
for i in range(0, len(relevant_texts), batch_size):
    batch_content = relevant_texts[i:i+batch_size]
    summaries = summarize_content_with_bart(batch_content, modified_query, max_length=100, min_length=30, num_beams=6)
    summarized_texts.extend(summaries)

final_combined_text = ' '.join(summarized_texts)

print("Query-Focused Combined Summary:")
print(final_combined_text)


Query-Focused Combined Summary:
Digital forensics deals with the analysis of artifacts on all types of digital devices. One of the most prevalent analysis techniques performed is that of the registry hives contained in microsoft windows operating systems. The security intelligence report sir is an investigation of the current threat landscape. It analyzes exploits vulnerabilities and malware based on data from over 600 million systems worldwide as well as internet services. The latest edition of blackberry s quarterly cybersecurity benchmarking guide covering events between march and may 2023 provides new information for the cybersecurity industry worldwide. Based on a detailed geopolitical analysis blackberry observed and stopped 1 5 million attacks within the 90 day period. The healthcare and financial services industries continue to be among the most targeted sectors.  symantec provide overview and analysis of the year in global threat activity via its internet security threat repor

In [ ]:
for text in summarized_texts:
    print(len(text))
    print()
    print(text)
    print('=================================')
    print()

223

Digital forensics deals with the analysis of artifacts on all types of digital devices. One of the most prevalent analysis techniques performed is that of the registry hives contained in microsoft windows operating systems.

222

The security intelligence report sir is an investigation of the current threat landscape. It analyzes exploits vulnerabilities and malware based on data from over 600 million systems worldwide as well as internet services.

404

The latest edition of blackberry s quarterly cybersecurity benchmarking guide covering events between march and may 2023 provides new information for the cybersecurity industry worldwide. Based on a detailed geopolitical analysis blackberry observed and stopped 1 5 million attacks within the 90 day period. The healthcare and financial services industries continue to be among the most targeted sectors.

402

 symantec provide overview and analysis of the year in global threat activity via its internet security threat report istr wi

-------------------------------------------------------------------------------------

-------------------------------------------------------------------------------

# **Module 7 (Response)**

In [ ]:
!pip install sentence-transformers
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
import json
import os

config_path = '/content/drive/MyDrive/1/Trained model/BERT_five_main/config.json'
with open(config_path, 'r') as f:
    config_data = json.load(f)

if config_data.get('early_stopping') is None:
    config_data['early_stopping'] = True
if config_data.get('num_beams') is None:
    config_data['num_beams'] = 6


with open(config_path, 'w') as f:
    json.dump(config_data, f)

print("Config file updated successfully!")


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

load_directory = '/content/drive/MyDrive/1/Trained model/BERT_five_main'

model = BartForConditionalGeneration.from_pretrained(load_directory)
tokenizer = BartTokenizer.from_pretrained(load_directory)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("Model and tokenizer loaded successfully!")


In [ ]:
input_text = " ".join(summarized_texts)

prompt = modified_query + " " + input_text

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)

generated_ids = model.generate(
    inputs['input_ids'],
    max_length=600,
    min_length=400,
    num_beams=6,
    no_repeat_ngram_size=3,
    repetition_penalty=2.0,
    length_penalty=1.5,
    early_stopping=False
)

response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

formatted_response = response
print(f"Generated Response: {formatted_response}")

Punchuation:

In [ ]:
!pip install deepmultilingualpunctuation

In [ ]:
from deepmultilingualpunctuation import PunctuationModel
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

punctuation_model = PunctuationModel()

bart_response = formatted_response

sentences = sent_tokenize(bart_response)

punctuated_sentences = [punctuation_model.restore_punctuation(sentence) for sentence in sentences]

punctuated_response = " ".join(punctuated_sentences)
print(f"Punctuated Response: {punctuated_response}")

def capitalize_sentences(text):
    sentences = sent_tokenize(text)
    capitalized_sentences = [sentence.capitalize() for sentence in sentences]
    return " ".join(capitalized_sentences)

final_response = capitalize_sentences(punctuated_response)
print(f"Final Response: {final_response}")

----------------------------------------------------------------------------------------

# **Module 8 (Link and Summary)**

In [ ]:
#initially
summary = 'no'

In [ ]:
if summary == 'no':
        for l in range(1, 11):
            print(f"Link {l}: {top_10_contents_df['Link'].iloc[l-1]}")

Link 1: https://thehackernews.com/2011/09/registry-decoder-digital-forensics-tool.html
Link 2: https://thehackernews.com/2011/05/microsoft-release-security-intelligence.html
Link 3: https://thehackernews.com/2023/08/malware-unleashed-public-sector-hit-in.html
Link 4: https://thehackernews.com/2012/10/400-million-new-variants-of-malware.html
Link 5: https://thehackernews.com/2023/08/numbers-dont-lie-exposing-harsh-truths.html
Link 6: https://thehackernews.com/2010/10/panda-security-brazilian-army-join.html
Link 7: https://thehackernews.com/2013/02/mandiant-revealed-chinese-apt1-cyber.html
Link 8: https://thehackernews.com/2013/06/Comment-Crew-Chinese-Hackers.html
Link 9: https://thehackernews.com/2023/02/the-future-of-network-security.html
Link 10: https://thehackernews.com/2011/11/crysys-duqu-detector-open-source.html


Say user wants to cheack summary of the link. so cliking on the link summary

In [ ]:
summary = 'yes'

if summary == 'yes':
        for l, t in zip(range(1, 11), range(10)):
            print(f"Link {l}: {top_10_contents_df['Link'].iloc[l-1]}")
            print(f"Summary {t+1}: {summarized_texts[t]}")
            print('========================================================================')
            print()

Link 1: https://thehackernews.com/2011/09/registry-decoder-digital-forensics-tool.html
Summary 1: Digital forensics deals with the analysis of artifacts on all types of digital devices. One of the most prevalent analysis techniques performed is that of the registry hives contained in microsoft windows operating systems.

Link 2: https://thehackernews.com/2011/05/microsoft-release-security-intelligence.html
Summary 2: The security intelligence report sir is an investigation of the current threat landscape. It analyzes exploits vulnerabilities and malware based on data from over 600 million systems worldwide as well as internet services.

Link 3: https://thehackernews.com/2023/08/malware-unleashed-public-sector-hit-in.html
Summary 3: The latest edition of blackberry s quarterly cybersecurity benchmarking guide covering events between march and may 2023 provides new information for the cybersecurity industry worldwide. Based on a detailed geopolitical analysis blackberry observed and stop

--------------------------------------------------------------------------------

# **Module 9 (Response History)**

In [ ]:
import os

In [ ]:
#initial CSV file path
response_history = '/content/drive/MyDrive/1/query_response.csv'

def append_to_csv(modified_query, input_text, generated_text, cat_label, response_history):
    if os.path.exists(response_history):

        dummy_df = pd.read_csv(response_history)

        if 'SL_no' not in dummy_df.columns:
            dummy_df.insert(0, 'SL_no', range(1, len(dummy_df) + 1))

        next_sl_no = dummy_df['SL_no'].max() + 1

    else:
        dummy_df = pd.DataFrame(columns=['SL_no', 'user_query', 'summarized_texts', 'response', 'label'])
        next_sl_no = 1

    new_row = pd.DataFrame({
        'SL_no': [next_sl_no],
        'user_query': [modified_query],
        'summarized_texts': [input_text],
        'response': [generated_text],
        'label': [predicted_initial_label]
    })

    dummy_df = pd.concat([dummy_df, new_row], ignore_index=True)

    dummy_df.to_csv(response_history, index=False)


append_to_csv(modified_query, input_text, generated_text, predicted_initial_label, response_history)


-------------------------------------------------------------------------------------

In [ ]:
previous_query = modified_query
modified_query = ""

In [ ]:
session_duration(session_start_time)

Active session duration: 97 seconds


# **=====================================================================================**

## **Module 10 (Real time based):**

In [ ]:
scrapped_data = pd.read_csv('/content/drive/MyDrive/1/Scrapped datasets/scrapped_data_set.csv')
main_data = pd.read_csv('/content/drive/MyDrive/1/CSV/data_set.csv')

In [ ]:
print(main_data.shape)
print(scrapped_data.shape)

**Pre-proccessing**

In [ ]:
print("Original Scraped Data Columns:", scrapped_data.columns)
scrapped_data.columns = scrapped_data.columns.str.strip()

if scrapped_data.columns[0] == '' or scrapped_data.columns[0] == 'Unnamed: 0':
    scrapped_data.rename(columns={scrapped_data.columns[0]: 'SL no'}, inplace=True)

scrapped_data = scrapped_data.rename(columns={"Category": "Double_Category"})
print("Cleaned Scraped Data Columns:", scrapped_data.columns)

In [ ]:
scrapped_data['Priority_Category'] = ''
scrapped_data['Final_Category'] = ''
scrapped_data['Category'] = ''

scrapped_data = scrapped_data[['SL no', 'Title', 'Link', 'Date', 'Double_Category', 'Priority_Category', 'Category', 'Final_Category', 'Content']]

**K-Means Clustering:**

In [ ]:
!pip install sentence_transformers

In [ ]:
import torch
import torch.nn as nn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import pandas as pd

In [ ]:
scrapped_data['Text'] = scrapped_data['Title'] + ' ' + scrapped_data['Content']

model = SentenceTransformer('all-MiniLM-L6-v2')
X_embeddings = model.encode(scrapped_data['Text'], convert_to_tensor=True)

X_tensor = torch.tensor(X_embeddings.cpu().detach().numpy(), dtype=torch.float32)

class EnhancedAutoencoderWithBatchNorm(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(EnhancedAutoencoderWithBatchNorm, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

input_dim = X_tensor.shape[1]
latent_dim = 4

autoencoder = EnhancedAutoencoderWithBatchNorm(input_dim, latent_dim)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

num_epochs = 100
for epoch in range(num_epochs):
    reconstructed = autoencoder(X_tensor)
    loss = criterion(reconstructed, X_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

with torch.no_grad():
    latent_features = autoencoder.encoder(X_tensor).numpy()

pca = PCA(n_components=2)
latent_features_pca = pca.fit_transform(latent_features)

kmeans = KMeans(n_clusters=4, random_state=42)
kmeans_labels = kmeans.fit_predict(latent_features_pca)

silhouette_avg_kmeans = silhouette_score(latent_features_pca, kmeans_labels)
dbi_kmeans = davies_bouldin_score(latent_features_pca, kmeans_labels)
print(f"K-Means Silhouette Score: {silhouette_avg_kmeans}")
print(f"K-Means Davies-Bouldin Index: {dbi_kmeans}")

tsne = TSNE(n_components=2, random_state=42)
latent_features_tsne = tsne.fit_transform(latent_features_pca)

plt.figure(figsize=(8, 6))
plt.scatter(latent_features_tsne[:, 0], latent_features_tsne[:, 1], c=kmeans_labels, cmap='plasma')
plt.title('K-Means on 1.3k real time Datapoint (4 clusters)')
plt.show()


In [ ]:
scrapped_data['Cluster'] = kmeans_labels

cluster_to_category = {
    0: 'Cyber Attack',
    1: 'Malware',
    2: 'Vulnerability',
    3: 'Data Breach'
}

scrapped_data['Final_Category'] = scrapped_data['Cluster'].map(cluster_to_category)

scrapped_data.drop(columns=['Text', 'Cluster'], inplace=True)

scrapped_data.to_csv('/content/drive/MyDrive/1/Scrapped datasets/kmeans_final_clusters.csv', index=False)

print("K-Means clustering completed and labels assigned.")

In [ ]:
klabeled_scrapped_data_set = pd.read_csv('/content/drive/MyDrive/1/Scrapped datasets/kmeans_final_clusters.csv')
print(klabeled_scrapped_data_set.shape)

In [ ]:
klabeled_scrapped_data_set.head()

------------------------------------------------------------------------------------------

In [ ]:
klabeled_scrapped_data_set.reset_index(drop=True, inplace=True)
main_data.reset_index(drop=True, inplace=True)

print("Scrapped Data Columns:", klabeled_scrapped_data_set.columns)
print("Main Data Columns:", main_data.columns)

In [ ]:
merged_data = pd.concat([klabeled_scrapped_data_set, main_data], ignore_index=True)

merged_data['SL no'] = range(1, len(merged_data) + 1)

**Save the scrapped data to main data**

In [ ]:
merged_data.to_csv('/content/drive/MyDrive/1/CSV/data_set.csv', index=False)

print("success")

In [ ]:
print(main_data.shape)

In [ ]:
main_data.head()